## タスク 3: Amazon SageMaker にモデルをデプロイする

このタスクでは、Amazon SageMaker を使用してトレーニング済みの機械学習モデルをデプロイし、予測を取得します。まず、モデルアーティファクトを Amazon S3 バケットにエクスポートします。次に、モデルアーティファクトから SageMaker Model オブジェクトを作成します。次に、Sagemaker Model、変換するデータの S3 バケットの場所、および結果を保存する S3 バケットの場所を指定して、バッチ変換ジョブを作成します。次に、バッチ変換ジョブを開始し、その進行状況を監視します。最後に、バッチ変換ジョブが完了したら、指定した S3 バケットから結果を取得します。結果には、入力データに対してトレーニング済みのモデルが行った推論が含まれています。

**注:** 課題を完了するのに問題が発生した場合は、ノートブックの下部にある<a href="#appendix" target="_self">**付録**</a>セクションにソリューションが用意されています。

### タスク 3.1: 環境を設定する

ラボの開始に必要となる、基本的な設定コードは以下に用意されています。パッケージのインストールと変数の作成のために、まずこれらのセルを読み込んで実行します。

In [ ]:
#ライブラリをインポートして変数値を設定する

import sys
import boto3
import os
import sagemaker
import time

sess = sagemaker.Session()

bucket = sess.default_bucket()
prefix = "lab3/batch-transform"
bucket_path = "s3://{}".format(bucket)

role = sagemaker.get_execution_role()

region = boto3.Session().region_name


### タスク 3.2: モデルを作成する

このタスクでは、事前トレーニング済みのモデルを取得して S3 にアップロードします。次に、SageMaker でモデルを作成します。このモデルは後で SageMaker バッチ変換ジョブを使用してデプロイされます。

#### 要件と設定

- **S3.Client.upload_file** API を使用して、モデルアーティファクトとテストデータを S3 バケットにアップロードします。

**upload_file** メソッドの使用方法の詳細については、[Upload a file to an S3 object](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/upload_file.html#upload-file) を参照してください。

In [ ]:
# モデルを Amazon S3 バケットにアップロードする
from sagemaker.session import Session
from sagemaker.image_uris import retrieve

sagemaker = boto3.client('sagemaker')
s3_client = boto3.client("s3")

#add_upload-model_code_here


#add_upload-dataset_code_here


**注:** モデルアーティファクトとテストデータを S3 バケットにアップロードする方法の詳細なコードについては、「*付録*」セクションの<a href="#task3-2-1" target="_self">**モデルとテストデータを S3 にアップロードする (タスク 3.2)**</a> を参照してください。

<a id="task3-2-continue"></a>

次に、上記のモデルアーティファクトから SageMaker でモデルオブジェクトを作成します。

#### 要件と設定

- **SageMaker.Client.create_model** API を使用して SageMaker でモデルを作成します。
- リクエストでは、次のオプションを使用します。
    - ModelName='telemarketing-model'
    - ExecutionRoleArn=role (SageMaker が使用する以前に作成された IAM ロール)
    - プライマリコンテナには、次のオプションを使用します。
        - Image= XGBoost container image URI (retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.5-1'))
        - ModelDataUrl=model_path (モデルアーティファクトが以前にアップロードされた S3 パス) 

**SageMaker.Client.create_model** メソッドの使用方法の詳細については、[Creating a model in SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_model.html) を参照してください。

In [ ]:
sagemaker = boto3.client('sagemaker')

#add_create-model_code_here

**注:** モデルの作成方法の詳細なコードについては、「*付録*」セクションの <a href="#task3-2-2" target="_self">**SageMaker でモデルを作成する (タスク 3.2)**</a> を参照してください。

<a id="task3-3"></a>

### タスク 3.3 バッチトランスフォーマーを作成する

このタスクでは、バッチトランスフォーマーを作成します。

**Transfomer** API の使用方法の詳細については、[Using Transformer API](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer) を参照してください。

#### 要件と設定

- **SageMaker.transformer.Transformer** API を使用して SageMaker でトランスフォーマーを初期化します。
- Parameters セクションで、次のオプションを使用します。
    - model_name="telemarketing-model"
    - instance_type="ml.m4.xlarge"
    - instance_count=1
    - assemble_with="Line"
    - strategy="MultiRecord"
    - accept="text/csv"
    - output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)

In [7]:
from sagemaker.transformer import Transformer

#add_create-batch-transformer_code_here
transformer = Transformer(
    
)

**注:** Transformer オブジェクトを作成する方法の詳細なコードについては、「*付録*」セクションの<a href="#task3-3-1" target="_self">**バッチトランスフォーマーオブジェクトを作成する (タスク3.3)**</a> を参照してください。

<a id="task3-4"></a>

### タスク 3.4: バッチ変換ジョブを実行する

このタスクでは、変換ジョブを実行します。トレーニングデータを渡して、バッチトランスフォーマーから予測を取得します。

**Transformer.transform** API の使用方法の詳細については、[Using Transform API](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer.transform) を参照してください。

#### 要件と設定

- **Transformer.transform** API を使用して SageMaker でトランスフォーマーを初期化します。
- Parameters セクションで、次のオプションを使用します。
    - batch_path (Amazon S3 で以前に作成した入力データの場所)
    - content_type="text/csv"
    - split_type="Line"
    - join_source="Input"

In [ ]:
transformer.transform(
 #add_run-batch-transform-job_code_here   
)

transformer.wait()

**注:** 変換ジョブの実行方法の詳細なコードについては、「*付録*」セクションの「<a href="#task3-4-1" target="_self">**バッチ変換ジョブを実行する (タスク 3.4)**</a> を参照してください。

<a id="task3-5"></a>

### タスク 3.5: S3 バケットに保存されている出力を確認する

このタスクでは、S3 のバッチ変換ジョブの出力を取得します。予測のリストが表示されるはずです。

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
!head test_data.csv.out

お疲れ様でした。Amazon SageMaker を使用してトレーニング済みの機械学習モデルをデプロイし、予測を取得できました。

### クリーンアップ

このノートブックを完了しました。ラボの次の部分に進むために、以下を実行してください。

- このノートブックファイルを閉じる。
- ラボのセッションに戻り、「**まとめ**」を続ける。

## その他のリソース

- [Deploy models for inference](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)
- [Using SageMaker APIs](https://sagemaker.readthedocs.io/en/stable/api/inference/index.html)

<a name="appendix" id="appendix"></a>

## 付録


<a name="task3-2-1" id="task3-2-1"></a>

### モデルとテストデータを S3 にアップロードする (タスク 3.2)

In [ ]:
# モデルを Amazon S3 バケットにアップロードする
from sagemaker.session import Session
from sagemaker.image_uris import retrieve

#session = boto3.Session()
sagemaker = boto3.client('sagemaker')
s3_client = boto3.client("s3")

#upload-model
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")  
model_path = f"s3://{bucket}/{prefix}/models/model.tar.gz"
print(model_path)

#upload-dataset
s3_client.upload_file(Filename="test_data.csv", Bucket=bucket, Key=f"{prefix}/test_data.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
batch_path = f"s3://{bucket}/{prefix}/test_data.csv"
print(batch_path)

このラボを続けるには、<a href="#task3-2-continue" target="_self">タスク 3.2</a> に戻ってください。

<a name="task3-2-2" id="task3-2-2"></a>

### SageMaker でモデルを作成する (タスク 3.2)

In [ ]:
sagemaker = boto3.client('sagemaker')

# コンテナイメージを取得する
image = retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.5-1')

primary_container = {"Image": image, "ModelDataUrl": model_path}

# モデルを設定する
create_model_response = sagemaker.create_model(
    ModelName='telemarketing-model', ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

<a href="#task3-3" target="_self">タスク 3.3</a> に戻って、このラボを続けます。

<a name="task3-3-1" id="task3-3-1"></a>

### バッチトランスフォーマーオブジェクトを作成する (タスク 3.3)

In [ ]:
from sagemaker.transformer import Transformer

#create-batch-transformer
transformer = Transformer(
    model_name="telemarketing-model",
    instance_type="ml.m4.xlarge",
    instance_count=1,
    assemble_with="Line",
    strategy="MultiRecord",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)
)

<a href="#task3-4" target="_self">タスク 3.4</a> に戻って、このラボを続けます。

<a name="task3-4-1" id="task3-4-1"></a>

### バッチ変換ジョブを実行する (タスク 3.4)


In [ ]:
#run-batch-transform-job
transformer.transform(batch_path, content_type="text/csv", split_type="Line", join_source="Input")

transformer.wait()

<a href="#task3-5" target="_self">タスク 3.5</a> に戻って、このラボを続けます。